In [22]:
import sys
sys.path.insert(1, '../unseen')

import numpy as np
from dask.distributed import Client, LocalCluster
import xclim.analog
import xarray as xr

import fileio

In [2]:
cluster = LocalCluster()
client = Client(cluster)

In [3]:
client

Client Scheduler: tcp://127.0.0.1:46339 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 28.00 GiB


## Read data

In [2]:
cafe_file = '/g/data/xv83/dbi599/pr_cafe-c5-d60-pX-f6_awap-additive-correction_19900501-19931101_A-DEC-sum_cafe-grid-TAS-POINT.zarr.zip'
ds_cafe = fileio.open_file(cafe_file)
ds_cafe          

<xarray.Dataset>
Dimensions:    (ensemble: 96, init_date: 8, lead_time: 11)
Coordinates:
  * ensemble   (ensemble) int64 1 2 3 4 5 6 7 8 9 ... 88 89 90 91 92 93 94 95 96
  * init_date  (init_date) object 1990-12-31 00:00:00 ... 1993-12-31 00:00:00
  * lead_time  (lead_time) int64 0 1 2 3 4 5 6 7 8 9 10
    time       (lead_time, init_date) object dask.array<chunksize=(11, 8), meta=np.ndarray>
Data variables:
    pr         (init_date, lead_time, ensemble) float64 dask.array<chunksize=(8, 11, 96), meta=np.ndarray>
Attributes:
    history:  Tue Jul 06 14:12:35 2021: /g/data/e14/dbi599/miniconda3/envs/un...

In [29]:
# Note: use_cftime=True when using xr.open_zarr breaks time slice selection

awap_file = '/g/data/xv83/dbi599/pr_awap_cafe-grid-TAS-POINT.zarr.zip'
ds_awap = xr.open_zarr(awap_file, consolidated=True)
ds_awap = ds_awap.sel({'time': slice('1980-01-01', '2000-12-31')})
ds_awap

<xarray.Dataset>
Dimensions:  (time: 21)
Coordinates:
  * time     (time) datetime64[ns] 1980-12-31 1981-12-31 ... 2000-12-31
Data variables:
    pr       (time) float64 dask.array<chunksize=(1,), meta=np.ndarray>
Attributes:
    history:  Tue Jul 06 13:54:07 2021: /g/data/e14/dbi599/miniconda3/envs/un...

## K-S test

In [8]:
fcst_stacked = ds_cafe.stack({'sample': ['ensemble', 'init_date', 'lead_time']})

In [13]:
fcst_stacked['pr']

<xarray.DataArray 'pr' (sample: 8448)>
dask.array<reshape, shape=(8448,), dtype=float64, chunksize=(8448,), chunktype=numpy.ndarray>
Coordinates:
    time       (sample) object dask.array<chunksize=(8448,), meta=np.ndarray>
  * sample     (sample) MultiIndex
  - ensemble   (sample) int64 1 1 1 1 1 1 1 1 1 1 ... 96 96 96 96 96 96 96 96 96
  - init_date  (sample) object 1990-12-31 00:00:00 ... 1993-12-31 00:00:00
  - lead_time  (sample) int64 0 1 2 3 4 5 6 7 8 9 10 ... 0 1 2 3 4 5 6 7 8 9 10
Attributes:
    bias_correction_method:  additive

In [8]:
#fcst_stacked['sample'] = np.arange(fcst_stacked['sample'].size)
#fcst_stacked

In [17]:
awap_faux_stacked = ds_awap.rename(time='sample')
#awap_faux_stacked['sample'] = np.arange(awap_faux_stacked['sample'].size)
awap_faux_stacked = awap_faux_stacked.chunk({'sample': -1})
awap_faux_stacked['pr']

<xarray.DataArray 'pr' (sample: 121)>
dask.array<rechunk-merge, shape=(121,), dtype=float64, chunksize=(121,), chunktype=numpy.ndarray>
Coordinates:
  * sample   (sample) object 1900-12-31 00:00:00 ... 2020-12-31 00:00:00
Attributes:
    analysis_time:                 2010-10-23 14:49:46 UTC
    analysis_version_number:       3.01
    cell_methods:                  time: sum
    frequency:                     daily
    grid_mapping:                  crs
    length_scale_for_analysis:     80.0
    long_name:                     Daily precipitation
    number_of_stations_reporting:  2757
    source:                        Interpolated surface observations
    standard_name:                 lwe_thickness_of_precipitation_amount
    units:                         mm d-1
    valid_range:                   [-1.0, 100000.0]

In [18]:
test = xclim.analog.spatial_analogs(target=awap_faux_stacked,
                                    candidates=fcst_stacked,
                                    dist_dim='sample',
                                    method='kolmogorov_smirnov')

In [19]:
test.values

array(0.10890152)

For a 2D KS-test I think you just include two variables in the Dataset instead of just one. For instance, at the moment `data_vars` only has one variable, but you could add another.

In [30]:
fcst_stacked.data_vars

Data variables:
    pr       (sample) float64 dask.array<chunksize=(8448,), meta=np.ndarray>

# Resampling (bootstrapping)

In [10]:
def random_resample(*objects, samples,
                    function=None, function_kwargs=None, replace=True):
    """
        Randomly resample from provided xarray objects and return the results of the subsampled dataset passed through \
        a provided function
                
        Parameters
        ----------
        *objects : xarray DataArray or Dataset
            Objects containing data to be resampled. The coordinates of the first object are used for resampling and the \
            same resampling is applied to all objects
        samples : dictionary
            Dictionary containing the dimensions to subsample, the number of samples and the continuous block size \
            within the sample. Of the form {'dim1': (n_samples, block_size), 'dim2': (n_samples, block_size)}. The first \
            object in objects must contain all dimensions listed in samples, but subsequent objects need not.
        function : function object, optional
            Function to reduced the subsampled data
        function_kwargs : dictionary, optional
            Keyword arguments to provide to function
        replace : boolean, optional
            Whether the sample is with or without replacement
                
        Returns
        -------
        sample : xarray DataArray or Dataset
            Array containing the results of passing the subsampled data through function
    """
    objects_sub = [obj.copy() for obj in objects]
    for dimension, (n_samples, block_size) in samples.items():
        n_blocks = int(n_samples / block_size)
        n_samples = n_blocks * block_size
        random_samples = [slice(x,x+block_size) for x in np.random.choice(len(objects_sub[0][dimension])-block_size+1, 
                                                                          size=n_blocks,
                                                                          replace=replace)]
        objects_sub = [xr.concat([obj.isel({dimension: random_sample}) for random_sample in random_samples],
                                 dim=dimension) 
                       if dimension in obj.dims else obj 
                       for obj in objects_sub]
    
    if function:
        if function_kwargs:
            res = tuple([function(obj, **function_kwargs) for obj in objects_sub])
        else:
            res = tuple([function(obj) for obj in objects_sub])
    else:
        res = tuple(objects_sub)
    
    if len(res) == 1:
        return res[0]
    else:
        return res
    

def n_random_resamples(*objects, samples, n_repeats, 
                       function=None, function_kwargs=None,
                       replace=True, with_dask=True):
    """
    Repeatedly randomly resample from provided xarray objects and return
    the results of the subsampled dataset passed through a provided function
                
    Parameters
    ----------
    objects : xarray DataArray or Dataset
        Objects containing data to be resampled.
        The coordinates of the first object are used for resampling and
        the same resampling is applied to all objects
    samples : dictionary
        Dictionary containing the dimensions to subsample, the number of samples and the continuous block size \
        within the sample. Of the form {'dim1': (n_samples, block_size), 'dim2': (n_samples, block_size)}
    n_repeats : int
        Number of times to repeat the resampling process
    function : function object, optional
        Function to reduced the subsampled data
    function_kwargs : dictionary, optional
        Keyword arguments to provide to function
    replace : boolean, optional
        Whether the sample is with or without replacement
    with_dask : boolean, optional
        If True, use dask to parallelize across n_repeats using dask.delayed
    write_to_disk
                
    Returns
    -------
    sample : xarray DataArray or Dataset
        Array containing the results of passing the subsampled data through function
    """

    if with_dask & (n_repeats > 1000):
        n_objects = itertools.repeat(objects[0], times=n_repeats)
        b = db.from_sequence(n_objects, npartitions=100)
        rs_list = b.map(random_resample, *(objects[1:]), 
                        **{'samples':samples, 'function':function, 
                           'function_kwargs':function_kwargs, 'replace':replace}).compute()
    else:              
        resample_ = dask.delayed(random_resample) if with_dask else random_resample
        rs_list = [resample_(*objects,
                             samples=samples,
                             function=function,
                             function_kwargs=function_kwargs,
                             replace=replace) for _ in range(n_repeats)] 
        if with_dask:
            rs_list = dask.compute(rs_list)[0]
    
    if len(objects) == 1:
        return xr.concat([r.unify_chunks() for r in rs_list], dim='k')
    else:
        return tuple([xr.concat([r.unify_chunks() for r in rs], dim='k') for rs in zip(*rs_list)])

In [ ]:
alpha = 5
n_repeats = 100
n_block = 1

fcst_moments = n_random_resamples(ds_stacked, 
                                  samples={'sample': (len(da_awap['time']), n_block)}, 
                                  n_repeats=n_repeats,
                                  function=get_first_four_moments,
                                  function_kwargs={'dim' : 'sample'})